In [308]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import operator
import sys
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree

In [309]:
data=pd.read_excel('tic_tack.xlsx')
test=pd.read_excel('test.xlsx')

In [310]:
sys.setrecursionlimit(2000)

In [311]:
data.head()

,tls,tms,trs,mls,mms,mrs,bls,bms,brs,result
0,x,x,x,x,o,o,x,o,o,positive
1,x,x,x,x,o,o,o,x,o,positive
2,x,x,x,x,o,o,o,o,x,positive
3,x,x,x,x,o,o,o,b,b,positive
4,x,x,x,x,o,o,b,o,b,positive


In [312]:
class tree:
    attr=''
    child={}

In [313]:
ex1=tree()
ex2=tree()
ex3=tree()
ex1.attr='naveen'
ex2.attr='kumar'
ex3.attr='don'
ex1.child[0]=ex2
ex1.child[1]=ex3
print(ex1.child[0].attr)
print(ex1.child[1].attr)

kumar
don


In [314]:
def attr_count_search(datas,attr_name,attr_value):
    pos,neg=0,0
    for i in range(0,datas[attr_name].count()):
        if(datas.loc[i,attr_name]==attr_value and datas.loc[i,'result']=='positive'):
            pos=pos+1
        if(datas.loc[i,attr_name]==attr_value and datas.loc[i,'result']=='negative'):
            neg=neg+1
    #print(pos,neg)
    return [pos,neg]
def calc_entropy(pos,neg):
    return ((-pos/(pos+neg))*math.log(pos/(pos+neg),2))+((-neg/(pos+neg))*math.log(neg/(pos+neg),2))
def class_entropy(datas):
    l1=list(datas['result'])
    pos=l1.count('positive')
    neg=l1.count('negative')
    #print(pos,neg)
    return calc_entropy(pos,neg)
def attr_entropy(datas,attr_name):
    l1=list(datas[attr_name])
    uniq=set(l1)
    attr_entr=0
    for i in uniq:
        val=attr_count_search(datas,attr_name,i)
        pos,neg=val[0],val[1]
        if(pos==0 or neg==0):
            entr=0
        else:
            entr=calc_entropy(pos,neg)
        attr_entr=attr_entr+(((pos+neg)/datas[attr_name].count())*entr)
    return attr_entr
def info_gain(datas):
    l1=datas.columns
    l2={}
    target=class_entropy(datas)
    #print(datas.columns)
    for i in range(len(l1)-1):
        #print(l1[i])
        attr_entr=attr_entropy(datas,l1[i])
        gain=target-attr_entr
        l2[l1[i]]=gain
    #print(l2)
    return max(l2.items(), key=operator.itemgetter(1))[0]
def getSubset(datas,attr,attr_value):
    new_data,j=pd.DataFrame(columns=datas.columns),0
    for i in range(0,datas['result'].count()):
        if(datas.loc[i,attr]==attr_value):
            row=datas[i:i+1]
            new_data=new_data.append(row)
            j=j+1
    del new_data[attr]
    return new_data
def tree_split(datas):
    high_gain=info_gain(datas)
    uniq=set(list(datas[high_gain]))
    print("info gain="+high_gain)
    node=tree()
    node.attr=high_gain
    temp=datas[:]
    del temp[high_gain]
    print(temp.columns)
    for i in uniq:
        new_data=getSubset(datas,high_gain,i)
        new_data=new_data.reset_index(drop=True)
        if(len(set(new_data['result']))==1):
            #print(i,set(new_data['result']))
            node.child[i]=list(set(new_data['result']))[0]
        else:
            node.child[i]=tree_split(new_data)
            #print(node.child[i])
    return node
#tree_split(data)

In [315]:
main=tree_split(data)

info gain=mms
Index(['tls', 'tms', 'trs', 'mls', 'mrs', 'bls', 'bms', 'brs', 'result'], dtype='object')
info gain=tls
Index(['tms', 'trs', 'mls', 'mrs', 'bls', 'bms', 'brs', 'result'], dtype='object')
info gain=brs
Index(['tms', 'trs', 'mls', 'mrs', 'bls', 'bms', 'result'], dtype='object')
info gain=tms
Index(['trs', 'mls', 'mrs', 'bls', 'bms', 'result'], dtype='object')
info gain=trs
Index(['mls', 'mrs', 'bls', 'bms', 'result'], dtype='object')
info gain=bms
Index(['mls', 'mrs', 'bls', 'result'], dtype='object')
info gain=bms
Index(['mls', 'mrs', 'bls', 'result'], dtype='object')
info gain=trs
Index(['mls', 'mrs', 'bls', 'bms', 'result'], dtype='object')
info gain=bls
Index(['mls', 'mrs', 'bms', 'result'], dtype='object')
info gain=bls
Index(['trs', 'mls', 'mrs', 'bms', 'result'], dtype='object')
info gain=bms
Index(['trs', 'mls', 'mrs', 'result'], dtype='object')
info gain=trs
Index(['mls', 'mrs', 'result'], dtype='object')
info gain=mrs
Index(['trs', 'mls', 'bms', 'result'], dtype='

In [198]:
def predict(l1,node):
    if(not node):
        return "not found"
    attr=node.attr
    print(node.attr,node.child[l1[attr]])
    res=node.child[l1[attr]]
    if(res=='positive' or res=='negative'):
        return res
    else:
        return predict(l1,res)

In [209]:
final={}
final['tls']='x'
final['tms']='b'
final['trs']='x'
final['mls']='b'
final['mms']='b'
final['mrs']='x'
final['bls']='o'
final['bms']='o'
final['brs']='o'
print(predict(final,main))

mms positive
positive


In [230]:
def printnode(nodes):
    print(nodes.attr)
    for i in nodes.child:
        #if(nodes.child[i]=='positive' or nodes.child[i]=='negative'):
        print(i,nodes.child[i])
        #else:
        #   printnode(nodes.child[i])

In [231]:
printnode(main)

mms
o <__main__.tree object at 0x0000016CC349EA90>
b positive
x <__main__.tree object at 0x0000016CC338C2B0>


In [244]:
xc=xc.child['o']
printnode(xc)

AttributeError: 'tree' object has no attribute 'left'

In [236]:
class trees:
    data=None
    left=None
    right=None

In [248]:
def insert(node,val):
    if(node==None):
        node=trees()
        node.data=val
        return node
    elif(node.data>val):
        node.left=insert(node.left,val)
    else:
        node.right=insert(node.right,val)
    return node

In [249]:
def printnode(node):
    if(node!=None):
        printnode(node.left)
        print(node.data)
        printnode(node.right)

In [250]:
root=trees()
root=insert(root,10)
root=insert(root,9)
root=insert(root,6)
root=insert(root,3)
root=insert(root,7)
printnode(root)

0
3
6
7
9
10


In [276]:
x=data.values[:,0:9]
y=data.values[:,9:10]
y1=[0]*len(y)
x1=[]
for i in x:
    x2=[]
    for j in i:
        if(j=='x'):
            x2.append(1)
        elif(j=='o'):
            x2.append(2)
        else:
            x2.append(3)
    x1.append(x2[:])
print(x1)
for i in range(len(y)):
    if(y[i][0]=='positive'):
        y1[i]=1
print(y1)

[[1, 1, 1, 1, 2, 2, 1, 2, 2], [1, 1, 1, 1, 2, 2, 2, 1, 2], [1, 1, 1, 1, 2, 2, 2, 2, 1], [1, 1, 1, 1, 2, 2, 2, 3, 3], [1, 1, 1, 1, 2, 2, 3, 2, 3], [1, 1, 1, 1, 2, 2, 3, 3, 2], [1, 1, 1, 1, 2, 3, 2, 2, 3], [1, 1, 1, 1, 2, 3, 2, 3, 2], [1, 1, 1, 1, 2, 3, 3, 2, 2], [1, 1, 1, 1, 3, 2, 2, 2, 3], [1, 1, 1, 1, 3, 2, 2, 3, 2], [1, 1, 1, 1, 3, 2, 3, 2, 2], [1, 1, 1, 2, 1, 2, 1, 2, 2], [1, 1, 1, 2, 1, 2, 2, 1, 2], [1, 1, 1, 2, 1, 2, 2, 2, 1], [1, 1, 1, 2, 1, 2, 2, 3, 3], [1, 1, 1, 2, 1, 2, 3, 2, 3], [1, 1, 1, 2, 1, 2, 3, 3, 2], [1, 1, 1, 2, 1, 3, 2, 2, 3], [1, 1, 1, 2, 1, 3, 2, 3, 2], [1, 1, 1, 2, 1, 3, 3, 2, 2], [1, 1, 1, 2, 2, 1, 1, 2, 2], [1, 1, 1, 2, 2, 1, 2, 1, 2], [1, 1, 1, 2, 2, 1, 2, 2, 1], [1, 1, 1, 2, 2, 1, 2, 3, 3], [1, 1, 1, 2, 2, 1, 3, 2, 3], [1, 1, 1, 2, 2, 1, 3, 3, 2], [1, 1, 1, 2, 2, 3, 1, 2, 3], [1, 1, 1, 2, 2, 3, 1, 3, 2], [1, 1, 1, 2, 2, 3, 2, 1, 3], [1, 1, 1, 2, 2, 3, 2, 3, 1], [1, 1, 1, 2, 2, 3, 3, 1, 2], [1, 1, 1, 2, 2, 3, 3, 2, 1], [1, 1, 1, 2, 2, 3, 3, 3, 3], [1, 1, 1, 2, 

In [277]:
X_train, X_test, y_train, y_test = train_test_split(x1,y1, test_size = 0.3, random_state = 100)

In [278]:
clf_entropy = DecisionTreeClassifier(criterion = "entropy", random_state = 100,
 max_depth=3, min_samples_leaf=5)
clf_entropy.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=5,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=100, splitter='best')

In [279]:
y_pred_en = clf_entropy.predict(X_test)
y_pred_en

array([1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1])

In [280]:
print("Accuracy is ", accuracy_score(y_test,y_pred_en)*100)

Accuracy is  75.0
